In [ ]:
from rekall.video_interval_collection_3d import VideoIntervalCollection3D
from rekall.interval_set_3d import Interval3D, IntervalSet3D
from vgrid_jupyter import VGridWidget
from esper.widget import intervals_with_metadata
from query.models import FaceGender

In [ ]:
video = Video.objects.get(id=1)

In [ ]:
facegender = list(FaceGender.objects.filter(face__frame__video=video).select_related('face', 'face__frame', 'gender'))
GENDER_CATEGORY = 0
gender_category = {
    GENDER_CATEGORY: {g.id: g.name for g in Gender.objects.all()}
}
intervals = [
    Interval3D(
        t=(f.face.frame.number / video.fps, (f.face.frame.number + 90) / video.fps),
        x=(f.face.bbox_x1, f.face.bbox_x2),
        y=(f.face.bbox_y1, f.face.bbox_y2),
        payload={'metadata': {
            'gender': {
                'type': 'Metadata_Categorical',
                'args': {
                    'category_type': GENDER_CATEGORY,
                    'category': f.gender.id
                }
            }
        }, 'draw_type': {
            'type': 'DrawType_Bbox'
        }}
    )
    for f in facegender]
vgrid_intervals = {'video_id': 1, 'interval_dict': {'test': IntervalSet3D(intervals).to_json()}}

In [ ]:
gender_colors = {
    Gender.objects.get(name='M').id: 'cyan',
    Gender.objects.get(name='F').id: 'red',
    Gender.objects.get(name='U').id: 'yellow',
}

VGridWidget(
    interval_blocks=[vgrid_intervals], 
    database={
        'tables': {            
            'videos': list(Video.objects.filter(id=video.id).values()),
            'gender': [{'color': gender_colors[g['id']], **g} for g in Gender.objects.all().values()]
        },
        'id_to_name': {GENDER_CATEGORY: 'gender'}
    })